In [1]:
import random

In [2]:
def transition_generator(maze, with_walls = True):
    transiciones = []

    n_rows, n_cols = len(maze), len(maze[0])

    for y in range(n_rows):
        for x in range(n_cols):
            # Si la casilla actual es un muro o una casilla 'x' (forzada a no demostracion),
            # no se puede guardar la transición
            if maze[y][x] != '#' and maze[y][x] != 'x':
                for a in range(4):
                    if a == 0:  # Mover hacia el norte
                        y1, x1 = max(0, y - 1), x
                    elif a == 1:  # Mover hacia el sur
                        y1, x1 = min(n_rows - 1, y + 1), x
                    elif a == 2:  # Mover hacia el oeste
                        y1, x1 = y, max(0, x - 1)
                    elif a == 3:  # Mover hacia el este
                        y1, x1 = y, min(n_cols - 1, x + 1)

                    # Si la casilla siguiente es un muro y el booleano es válido,
                    # se debe guardar la transición de la casilla actual a sí misma
                    if maze[y1][x1] == '#':
                        if with_walls:
                            transiciones.append([y, x, a, y, x])
                    # Si la casilla siguiente es una casilla 'x' no vamos a guardar ninguna transición
                    elif maze[y1][x1] == 'x':
                        pass
                    # Si la siguiente no es ninguna de las anteriores, se guarda la transición
                    else:
                        transiciones.append([y, x, a, y1, x1])

    return transiciones

In [3]:
def transitions_families_generator_66_v0(transiciones_iniciales, n):
    n_transiciones_iniciales = len(transiciones_iniciales)
    n_borradas = int((1 - 0.66) * n_transiciones_iniciales)
    
    nuevos_arrays = []

    for _ in range(n):
        # Crear una copia de las transiciones iniciales
        nuevas_transiciones = [sublista.copy() for sublista in transiciones_iniciales]

        # Eliminar aleatoriamente 'n_retenidas' sublistas
        sublistas_a_eliminar = random.sample(nuevas_transiciones, n_borradas)
        for sublista in sublistas_a_eliminar:
            nuevas_transiciones.remove(sublista)

        # Repetir aleatoriamente 'n_borradas' sublistas con reemplazo
        sublistas_a_repetir = random.choices(nuevas_transiciones, k=n_borradas)
        nuevas_transiciones.extend(sublistas_a_repetir)

        nuevos_arrays.append(nuevas_transiciones)

    return nuevos_arrays

In [4]:
def transitions_families_generator_66_v2(transiciones_iniciales, n):
    n_transiciones_iniciales = len(transiciones_iniciales)
    n_borradas = int((1 - 0.66) * n_transiciones_iniciales)
    
    nuevos_arrays = []

    for _ in range(n):
        # Crear una copia de las transiciones iniciales
        nuevas_transiciones = transiciones_iniciales.copy()

        # Eliminar aleatoriamente 'n_retenidas' transiciones
        transiciones_a_eliminar = random.sample(nuevas_transiciones, n_borradas)
        for t in transiciones_a_eliminar:
            nuevas_transiciones.remove(t)

        # Añadir transiciones aleatorias adicionales para completar la longitud
        transiciones_a_adicionar = random.sample(nuevas_transiciones, n_transiciones_iniciales - len(nuevas_transiciones))
        nuevas_transiciones.extend(transiciones_a_adicionar)

        nuevos_arrays.append(nuevas_transiciones)

    return nuevos_arrays


In [5]:
maze5x5 = { "starting_pos": [[0,0], [2,0], [4,0]],
             "maze":[
                    ['.', '.', '#', 'x', 'G'],#0            0
                    ['.', '.', '#', 'x', '.'],#1            ↑
                    ['.', '.', '.', '.', '.'],#2         2 ← → 3
                    ['.', 'x', '#', 'x', 'x'],#3            ↓
                    ['S', 'x', '#', 'x', 'x'],#4            1
                    # 0    1    2    3    4
                    ]
        }

In [6]:
shape = "5x5"

maze = maze5x5["maze"]

In [7]:
all_transitions = transition_generator(maze)

print("Número de transiciones generadas:", len(all_transitions))
n = 2  # Generar N nuevos arrays

Número de transiciones generadas: 44


In [8]:
new_transitions = transitions_families_generator_66_v2(all_transitions, n)

In [9]:
from collections import defaultdict

def count_original_transitions(all_transitions, new_transitions):
    # Aplanar la lista de sublistas
    flattened_new_transitions = [item for sublist in new_transitions for item in sublist]

    # Inicializar un diccionario para contar las transiciones originales
    counts = defaultdict(int)

    # Contar la frecuencia de cada transición original
    for transition in flattened_new_transitions:
        counts[tuple(transition)] += 1

    index = 1
    # Imprimir los resultados
    for original_transition in all_transitions:
        count = counts.get(tuple(original_transition), 0)
        print(f"{index} Transición: {original_transition}, Apariciones: {count}")
        index += 1

In [10]:
print(len(all_transitions))

for t in all_transitions:
    print(t)

print("-------------------")

for t in new_transitions[0]:
    print(t)

print("-------------------")
for t in new_transitions[1]:
    print(t)

count_original_transitions(all_transitions, new_transitions)

44
[0, 0, 0, 0, 0]
[0, 0, 1, 1, 0]
[0, 0, 2, 0, 0]
[0, 0, 3, 0, 1]
[0, 1, 0, 0, 1]
[0, 1, 1, 1, 1]
[0, 1, 2, 0, 0]
[0, 1, 3, 0, 1]
[0, 4, 0, 0, 4]
[0, 4, 1, 1, 4]
[0, 4, 3, 0, 4]
[1, 0, 0, 0, 0]
[1, 0, 1, 2, 0]
[1, 0, 2, 1, 0]
[1, 0, 3, 1, 1]
[1, 1, 0, 0, 1]
[1, 1, 1, 2, 1]
[1, 1, 2, 1, 0]
[1, 1, 3, 1, 1]
[1, 4, 0, 0, 4]
[1, 4, 1, 2, 4]
[1, 4, 3, 1, 4]
[2, 0, 0, 1, 0]
[2, 0, 1, 3, 0]
[2, 0, 2, 2, 0]
[2, 0, 3, 2, 1]
[2, 1, 0, 1, 1]
[2, 1, 2, 2, 0]
[2, 1, 3, 2, 2]
[2, 2, 0, 2, 2]
[2, 2, 1, 2, 2]
[2, 2, 2, 2, 1]
[2, 2, 3, 2, 3]
[2, 3, 2, 2, 2]
[2, 3, 3, 2, 4]
[2, 4, 0, 1, 4]
[2, 4, 2, 2, 3]
[2, 4, 3, 2, 4]
[3, 0, 0, 2, 0]
[3, 0, 1, 4, 0]
[3, 0, 2, 3, 0]
[4, 0, 0, 3, 0]
[4, 0, 1, 4, 0]
[4, 0, 2, 4, 0]
-------------------
[0, 0, 0, 0, 0]
[0, 0, 2, 0, 0]
[0, 0, 3, 0, 1]
[0, 1, 1, 1, 1]
[0, 1, 2, 0, 0]
[0, 1, 3, 0, 1]
[0, 4, 0, 0, 4]
[0, 4, 1, 1, 4]
[0, 4, 3, 0, 4]
[1, 0, 1, 2, 0]
[1, 0, 3, 1, 1]
[1, 1, 1, 2, 1]
[1, 1, 2, 1, 0]
[1, 1, 3, 1, 1]
[1, 4, 0, 0, 4]
[1, 4, 1, 2, 4]
[1, 4, 3, 1, 4]
[